In [1]:
import pandas as pd
from database.market import Market
from database.adatabase import ADatabase
from processor.processor import Processor as p
from datetime import datetime
from tqdm import tqdm
import numpy as np
from modeler_strats.universal_modeler import UniversalModeler

In [2]:
market = Market()
spec = ADatabase("spec_btc")
umod = UniversalModeler()

In [3]:
factors = []
factors.append("d1")
factors.append("d2")
factors.append("d3")
factors.append("rolling14")

In [4]:
included_columns=[
    "year",
    "week",
    "ticker",
    "adjclose",
    "y"
]

In [5]:
included_columns.extend(factors)

In [6]:
training_sets = []

In [9]:
prices = pd.read_csv("./csv_files/prices/BTC-USD.csv")
prices = p.column_date_processing(prices)
prices["year"] = [x.year for x in prices["date"]]
prices["quarter"] = [x.quarter for x in prices["date"]]
ticker_data = prices
ticker_data.sort_values("date",ascending=True,inplace=True)
ticker_data["adjclose"] = [float(x) for x in ticker_data["adjclose"]]
ticker_data = ticker_data.groupby(["year","week"]).mean().reset_index()
ticker_data["d1"] = ticker_data["adjclose"].pct_change(periods=1)
ticker_data["d2"] = ticker_data["d1"].pct_change(periods=1)
ticker_data["d3"] = ticker_data["d2"].pct_change(periods=1)
ticker_data["rolling14"] = ticker_data["adjclose"].rolling(window=14).mean()
ticker_data.dropna(inplace=True)
ticker_data["ticker"] = "BTC"
ticker_data["future"] = ticker_data["adjclose"].shift(-1)
ticker_data["delta"] = (ticker_data["future"] - ticker_data["adjclose"]) / ticker_data["adjclose"]
ticker_data["y"] = [x > 0 for x in ticker_data["delta"]]
ticker_data = ticker_data.replace([np.inf, -np.inf], np.nan).dropna()
ticker_data.dropna(inplace=True)
ticker_data = ticker_data[included_columns]
training_sets.append(ticker_data)

In [10]:
data = pd.concat(training_sets)

In [11]:
data = data.dropna()

In [12]:
data

,year,week,ticker,adjclose,y,d1,d2,d3,rolling14
13,2014,50,BTC,351.792712,False,-0.064348,-15.492858,19.339296,368.388483
14,2014,51,BTC,324.602997,False,-0.077289,0.201103,-1.012980,369.107745
15,2014,52,BTC,324.146572,False,-0.001406,-0.981807,-5.882119,362.485428
16,2015,1,BTC,293.639503,False,-0.094115,65.933264,-68.155001,354.338781
17,2015,2,BTC,281.316140,False,-0.041968,-0.554081,-1.008404,348.616607
...,...,...,...,...,...,...,...,...,...
441,2023,6,BTC,22299.014230,True,-0.041856,-6.179370,6.118892,19225.297426
442,2023,7,BTC,23642.106306,True,0.060231,-2.439014,-0.605297,19660.973556
443,2023,8,BTC,23905.221819,False,0.011129,-0.815226,-0.665756,20179.092198
444,2023,9,BTC,22991.969308,False,-0.038203,-4.432715,4.437405,20651.753520


In [13]:
years = 4
for modeler in [umod]:
    for year in tqdm(range(2019,2023)):
        training_slice = data[(data["year"]<year) & (data["year"] >= year - years)].reset_index(drop=True)
        prediction_set = data[data["year"]==year].reset_index(drop=True)
        set_name = "sim"
        stuff = modeler.recommend_classification_model(training_slice,factors,multioutput=False)
        stuff["training_years"] = years

  0%|                                                                                                                                                              | 0/4 [00:00<?, ?it/s]

Epoch 1/10
5/5 [==============================] - 0s 2ms/step - loss: 382.6129 - accuracy: 0.5541
Epoch 2/10
5/5 [==============================] - 0s 1ms/step - loss: 328.6661 - accuracy: 0.5414
Epoch 3/10
5/5 [==============================] - 0s 1ms/step - loss: 297.6216 - accuracy: 0.5541
Epoch 4/10
5/5 [==============================] - 0s 1ms/step - loss: 312.3242 - accuracy: 0.5223
Epoch 5/10
5/5 [==============================] - 0s 1ms/step - loss: 292.4773 - accuracy: 0.5032
Epoch 6/10
5/5 [==============================] - 0s 1ms/step - loss: 297.3690 - accuracy: 0.5541
Epoch 7/10
5/5 [==============================] - 0s 1ms/step - loss: 247.8511 - accuracy: 0.5669
Epoch 8/10
5/5 [==============================] - 0s 1ms/step - loss: 339.6029 - accuracy: 0.4777
Epoch 9/10
5/5 [==============================] - 0s 1ms/step - loss: 239.6805 - accuracy: 0.6115
Epoch 10/10
2/2 [==============================] - 0s 1ms/step


 25%|█████████████████████████████████████▌                                                                                                                | 1/4 [00:24<01:12, 24.11s/it]

Epoch 1/10
5/5 [==============================] - 0s 1ms/step - loss: 270.8972 - accuracy: 0.4679
Epoch 2/10
5/5 [==============================] - 0s 1ms/step - loss: 290.8687 - accuracy: 0.4487
Epoch 3/10
5/5 [==============================] - 0s 1ms/step - loss: 266.1268 - accuracy: 0.5000
Epoch 4/10
5/5 [==============================] - 0s 1ms/step - loss: 279.4962 - accuracy: 0.5128
Epoch 5/10
5/5 [==============================] - 0s 1ms/step - loss: 250.9156 - accuracy: 0.5128
Epoch 6/10
5/5 [==============================] - 0s 1ms/step - loss: 238.9463 - accuracy: 0.5641
Epoch 7/10
5/5 [==============================] - 0s 1ms/step - loss: 262.9842 - accuracy: 0.5128
Epoch 8/10
5/5 [==============================] - 0s 1ms/step - loss: 211.1204 - accuracy: 0.4679
Epoch 9/10
5/5 [==============================] - 0s 1ms/step - loss: 169.1870 - accuracy: 0.5769
Epoch 10/10
2/2 [==============================] - 0s 2ms/step


 50%|███████████████████████████████████████████████████████████████████████████                                                                           | 2/4 [00:51<00:51, 25.79s/it]

Epoch 1/10
5/5 [==============================] - 0s 2ms/step - loss: 492.3776 - accuracy: 0.4872
Epoch 2/10
5/5 [==============================] - 0s 1ms/step - loss: 502.9213 - accuracy: 0.4872
Epoch 3/10
5/5 [==============================] - 0s 1ms/step - loss: 532.0969 - accuracy: 0.4808
Epoch 4/10
5/5 [==============================] - 0s 1ms/step - loss: 482.9414 - accuracy: 0.4936
Epoch 5/10
5/5 [==============================] - 0s 1ms/step - loss: 479.2236 - accuracy: 0.4808
Epoch 6/10
5/5 [==============================] - 0s 1ms/step - loss: 427.7989 - accuracy: 0.5192
Epoch 7/10
5/5 [==============================] - 0s 1ms/step - loss: 477.2938 - accuracy: 0.5064
Epoch 8/10
5/5 [==============================] - 0s 1ms/step - loss: 493.4785 - accuracy: 0.5192
Epoch 9/10
5/5 [==============================] - 0s 1ms/step - loss: 439.7801 - accuracy: 0.4936
Epoch 10/10
2/2 [==============================] - 0s 1000us/step


 75%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████▌                                     | 3/4 [01:18<00:26, 26.40s/it]

Epoch 1/10
5/5 [==============================] - 0s 2ms/step - loss: 1359.9952 - accuracy: 0.4522
Epoch 2/10
5/5 [==============================] - 0s 2ms/step - loss: 1227.6227 - accuracy: 0.5287
Epoch 3/10
5/5 [==============================] - 0s 1ms/step - loss: 1168.4987 - accuracy: 0.4522
Epoch 4/10
5/5 [==============================] - 0s 2ms/step - loss: 1553.5315 - accuracy: 0.4841
Epoch 5/10
5/5 [==============================] - 0s 1ms/step - loss: 1135.7750 - accuracy: 0.5032
Epoch 6/10
5/5 [==============================] - 0s 1ms/step - loss: 1493.4648 - accuracy: 0.5223
Epoch 7/10
5/5 [==============================] - 0s 1ms/step - loss: 1709.8225 - accuracy: 0.3949
Epoch 8/10
5/5 [==============================] - 0s 2ms/step - loss: 923.2621 - accuracy: 0.5478
Epoch 9/10
5/5 [==============================] - 0s 1ms/step - loss: 1388.9606 - accuracy: 0.4586
Epoch 10/10
2/2 [==============================] - 0s 2ms/step


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4/4 [01:42<00:00, 25.71s/it]


In [ ]:
stuff["model"] = [pickle.dumps(x) for x in stuff["model"]]

In [ ]:
spec.connect()
models = spec.store("models",stuff)
spec.disconnect()